In [30]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo
import sqlalchemy
from sqlalchemy import create_engine
import pymysql
import mysql.connector
import json
from pandas.io.json import json_normalize
from config import mongoConn, remote_db_endpoint, remote_db_port, remote_db_name, remote_db_user, remote_db_pwd
from ast import literal_eval

In [5]:
pymysql.install_as_MySQLdb() 
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
alchemy_conn = engine.connect()

In [34]:
def generate_user_service_csv(): 
    df = pd.read_sql("SELECT Service_Id id , Service_Name SNAME From streamingservices", con=alchemy_conn)
    serviceDct={}
    for i, row in df.iterrows():
        serviceDct[row['SNAME'].replace(' ','_').upper()] = row['id']
    serviceDct['AMAZON']=37
    serviceDct['YOUTUBE']=80  
    
    Ndf = pd.read_csv("data/NIESEN_VOD_VIEWING_HABITS-V4.csv")
    Ndf = Ndf.loc[:, ['username', 'services', 'freq']]
    lst = []
    for i, row in Ndf.iterrows(): 
        service_dict = literal_eval(row.services) 
        slist = []
        for k, v in service_dict.items(): 
            try:
                k = k.upper().replace(' ','_')
                slist.append(serviceDct[k])                
            except:
                pass            
        lst.append({
            'username': row.username
            , 'services':slist
            , 'freq':row.freq.split('-')[0]
        })  
     
    df = pd.DataFrame(lst)
    df.to_csv('data/user_service.csv', index=False)
generate_user_service_csv()

In [45]:
df = pd.read_csv('data/user_service.csv')
df[0:2]
df_synth = df

In [ ]:
pymysql.install_as_MySQLdb() 
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}") 
conn = engine.connect()

In [47]:
df_up = pd.read_sql("SELECT User_Id, User_Name username From user_profile", con=alchemy_conn)
df_up[0:2]

,User_Id,username
0,16,fabian_couture
1,17,winford_underwood


In [54]:
df_merged = pd.merge(df_synth , df_up, left_on='username',  right_on='username', suffixes=('','_'))
df_merged.to_csv('data/user_service_mapping.csv', index=False)
df_merged[0:2]

,username,services,freq,User_Id
0,fabian_couture,[30],1,16
1,winford_underwood,"[80, 12]",2,17


In [67]:
from ast import literal_eval 
df_usm = pd.read_csv('data/user_service_mapping.csv')
df_usm  
lst = df_usm.loc[1, ['services']]   
lst = literal_eval(lst['services'])
for item in lst:
    print(item)


80
12


In [18]:
sql = '''
SELECT user_name, ups.User_Id uid, ups.Service_Id, sts.Service_name  FROM user_profile up
LEFT JOIN user_profile_services ups ON up.User_Id = ups.User_Id
LEFT JOIN streamingservices sts ON sts.Service_Id = ups.Service_Id
    '''
df = pd.read_sql(sql, con=conn)
_json = df.to_json(orient='records')
json.dumps(_json, indent=2)
 

'"[{\\"user_name\\":\\"fabian_couture\\",\\"uid\\":16,\\"Service_Id\\":30,\\"Service_name\\":\\"Hulu\\"},{\\"user_name\\":\\"fabian_couture\\",\\"uid\\":16,\\"Service_Id\\":34,\\"Service_name\\":\\"Netflix\\"},{\\"user_name\\":\\"winford_underwood\\",\\"uid\\":17,\\"Service_Id\\":30,\\"Service_name\\":\\"Hulu\\"},{\\"user_name\\":\\"winford_underwood\\",\\"uid\\":17,\\"Service_Id\\":34,\\"Service_name\\":\\"Netflix\\"},{\\"user_name\\":\\"darell_akers\\",\\"uid\\":18,\\"Service_Id\\":30,\\"Service_name\\":\\"Hulu\\"},{\\"user_name\\":\\"darell_akers\\",\\"uid\\":18,\\"Service_Id\\":34,\\"Service_name\\":\\"Netflix\\"},{\\"user_name\\":\\"don_macon\\",\\"uid\\":19,\\"Service_Id\\":30,\\"Service_name\\":\\"Hulu\\"},{\\"user_name\\":\\"don_macon\\",\\"uid\\":19,\\"Service_Id\\":34,\\"Service_name\\":\\"Netflix\\"},{\\"user_name\\":\\"glenda_corona\\",\\"uid\\":20,\\"Service_Id\\":30,\\"Service_name\\":\\"Hulu\\"},{\\"user_name\\":\\"glenda_corona\\",\\"uid\\":20,\\"Service_Id\\":34,\\"Ser